#Junhao Zhang "Freddie" ID#11356533 Cpts355 SSPS

In [1475]:
# For tokenizing we'll use the re package for Regular Expressions. (copy this to your own sps.ipynb notebook) 
import re

def tokenize(s):
    return re.findall("/?[a-zA-Z][a-zA-Z0-9_]*|[-]?[0-9]+|[}{]+|%.*|[^ \t\n]", s)

In [ ]:
# See what tokenize does
print (tokenize(
"""
/fact {
dictz
begin
/n exch def
n 2 lt
{1}
{n -1 add fact n mul }
ifelse
end
}def
5 fact =
"""))

In [ ]:
# The it argument is an iterator that returns left or right parenthesis characters. 
# The sequence of return characters should represent a string of properly nested 
# parentheses pairs, from which the leading '(' has already been removed. If the 
# parenteses are not properly nested, returns False.
def groupMatching(it):
    res = []
    for c in it:
        if c=='}':
            return res
        else:
            # note how we use a recursive call to group the inner
            # matching parenthesis string and append it as a whole
            # to the list we are constructing.
            res.append(groupMatching(it))
    return False

# function to turn a string of properly nested parentheses
# into a list of properly nested lists.
def group(s):
    if s[0]=='{':
        return groupMatching(iter(s[1:]))
    else: return False                  # If it starts with ')' it is not properly nested

def _groups(tokens):
    code = []
    cur = ""
    i = 0
    
    while i < len(tokens):
        x = tokens[i]
        if x == '{':
            cur = cur +'{'
            i = i + 1
            tmp = _groups(tokens[i:])

            if not tmp:
                return False
            elif type(tmp) == tuple:
                i = i + int(tmp[1])
                code.append(tmp[0])
                cur = cur + str(tmp[2])
            else:
                i = i + len(tmp)
                code.append(tmp)
            
        elif x=='}':
            return (code,i, cur+'}')
        else:
            code.append(x)
        i = i + 1

    if len(cur) > 0 and cur[0] == '{':
        if group(cur) == False:
            return False
        
    return code

_groups(tokenize(
"""
/fact {
dictz
begin
/n exch def
n 2 lt
{1}
{n -1 add fact n mul }
ifelse
end
}def
5 fact =
"""))


In [ ]:
# In this cell, write your parsing code; it takes a list of tokens produced by tokenize
# and returns a code array; copy this cell into your sps.ipynb notebook and write the 
# necessary code. Of course you may create additional functions to help you write parse()
#
def parse(tokens):
    tmp = _groups(tokens)
    if not tmp or type(tmp) == tuple:
        return False
    else:
        return tmp

parse(tokenize(
"""
/fact {
dictz
begin
/n exch def
n 2 lt
{1}
{n -1 add fact n mul }
ifelse
end
}def
5 fact =
"""))


In [1479]:
def looklookup3(name, lists):
    if type(name) is not int and type(name) is not list:
        if type(name) == str:
            if name != "true" and name!= "false":
                if not digitOrMinus(name):
                        for i in range(len(lists) - 1, -1, -1):
                            if name in lists[i][3]:
                                return (lists[i][3][name],lists[i][1], lists[i][2]) 
                        return False
                return digitOrMinus(name)
            else:
                return name
        return False
    else:
        return name

def lookup(name, lists=0, dyna = True):

    if lists > 0:
            od = sorted(seq)
            for x in reversed(od):
                for y in seq[x]:
                    if lists == y[2] and name in y[3]:
                        return y[3][name]
            tmp = looklookup3(name, dictstack[:lists])
            if type(name) == str and type(tmp) == str:
                return looklookup3(tmp, dictstack[:lists])
            return tmp
    
    tmp = looklookup3(name, dictstack)
    if type(name) == str and type(tmp) == str:
        return looklookup3(tmp, dictstack)
    return tmp

In [1480]:
# The operand stack: define the operand stack and its operations in this notebook cell
opstack = []
seq = {}
# now define functions to push and pop values on the opstack according to your decision about which
# end should be the hot end.

def opPop():
    tmp = opstack[-1]
    del opstack[-1]
    return tmp

def opPush(value):
    if value == "0":
        opstack.append(0)
    elif type(value) == str:
        if value[0] == '/':
            opstack.append(value)
        elif lookup(value) != False:
            opstack.append(lookup(value))
        else:
            print("Error from opPush, for " + str(value))
            errorHandler0()
    elif type(value) == list or type(value) == int:
        opstack.append(value)
    else:
        print("Error from opPush, for " + str(value))
        errorHandler0()
    return

#Handle the error
def errorHandler2(x, b):
    opPush(b)
    opPush(x)
    print("Error input: violate the syntax for those two elements")

def errorHandler1(x):
    opPush(x)
    print("Error input: violate the syntax that one element")

def errorHandler0():
    print("Error input: violate the syntax for zero element")
    
#Check enough elements
def enoughInSt1():
    if len(opstack) > 0:
        return True
    else:
        print("No enought elements! Required 1 but only "+str(len(opstack))+" exist(s)")
        return False
    
def enoughInSt2():
    if len(opstack) > 1:
        return True
    else:
        print("No enought elements! Required 2 but only "+str(len(opstack))+" exist(s)")
        return False
    
#Check the status if it is integer in "str" type
def digitOrMinus(num):
    if type(num) == str or type(num)==int:
        if str(num).isdigit():
            return int(num)
        elif str(num[1:]).isdigit():
            if num[0] == "-":
                return 0 - int(num[1:])
            elif num[0] == "+":
                return int(num[1:])
            else:
                return False
        else:
            return False
    else:
        return False

# Remember that there is a Postscript operator called "pop" so we choose different names for these functions.


In [1481]:
# The dictionary stack: define the dictionary stack and its operations in this cell
dictstack = []
# now define functions to push and pop dictionaries on the dictstack, to define name, and to lookup a name

def dictPop():
    del dictstack[-1]
    return

def dictPush():
    dictstack.append(opPop())
    return

def define(name,value,dy=0,st=0):
    tmp = {}
    _name = ""
    if type(name) == str and name[0] == '/':
        _name = name[1:]
    else:
        _name = name
    tmp[_name] = value
    dictstack.append((dy,len(dictstack),st,tmp))
    return 

In [1482]:
# Arithmetic operators: define all the arithmetic operators in this cell -- add, sub, mul, div, eq, lt, gt
def add():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            opPush(lookup(x)+lookup(b))

        return

def sub():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            opPush(lookup(b)-lookup(x))

        return

def mul():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            opPush(lookup(x)*lookup(b))

        return

def div():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            if lookup(x) != 0:
                opPush(lookup(b)/lookup(x))
            else:
                errorHandler2(x, b)
        return

def gt():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            if lookup(b) > lookup(x):
                opPush("true")
            else:
                opPush("false")

        return

def lt():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b) or type(x) is not int or type(b) is not int:
            errorHandler2(x, b)
            print("Error!")
        else:
            if lookup(b) < lookup(x):
                opPush("true")
            else:
                opPush("false")
        return

def eq():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if not lookup(x) or not lookup(b):
            errorHandler2(x, b)
            print("Error!")
        else:
            if lookup(b) == lookup(x):
                opPush("true")
            else:
                opPush("false")

        return

In [1483]:
# Boolean operators: define all the boolean operators in this cell -- and, or, not

def psAnd():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        m = lookup(x)
        n = lookup(b)
        
        if not m or not b:
            errorHandler2(x, b)
            
        else:
            if m == "true":
                m = True
            elif m == "false":
                m = False
            else:
                errorHandler2(x, b)
                return

            if n == "true":
                n = True
            elif n == "false":
                n = False
            else:
                errorHandler2(x, b)
                return
            if type(m) is bool and type(n) is bool:
                if m and n:
                    
                    opPush("true")
                else:
                    
                    opPush("false")
            else:
                errorHandler2(x,b)

        return


def psOr():
    if enoughInSt2():
        x = opPop()
        b = opPop()
        m = lookup(x)
        n = lookup(b)
        
        if not m or not b:
            errorHandler2(x, b)
            
        else:
            if not digitOrMinus(m):
                if m == "true":
                    m = True
                elif m == "false":
                    m = False
                else:
                    errorHandler2(x, b)
                    return
            else:
                m = digitOrMinus(m)
            if not digitOrMinus(n):
                if n == "true":
                    n = True
                elif n == "false":
                    n = False
                else:
                    errorHandler2(x, b)
                    return
            else:
                n = digitOrMinus(n)

            if type(m) == bool and type(n) == bool:
                if m or n:
               
                    opPush("true")
                else:
             
                    opPush("false")
            elif type(m) == int and type(n) == int:
                opPush(m+n)
            else:
                print("Unexpected error")
        return

def psNot():
    if enoughInSt2():
        x = opPop()
        m = lookup(x)
        
        if not m:
            errorHandler1(x)
            
        else:
            if not digitOrMinus(m):
                if m == "true":
                    m = True
                elif m == "false":
                    m = False
                else:
                    errorHandler1(x)
                    return
            else:
                m = digitOrMinus(m)

            if type(m) == bool:
                if m:
           
                    opPush("false")
                else:
       
                    opPush("true")
            elif type(m) == int:
                #Accordance to Ghostscript
                opPush(0 - m - 1)
            else:
                print("Unexpected error")

        return

In [1484]:
# Define the stack manipulation operators in this cell: dup, exch, pop
def dup():
    if enoughInSt1():
        x = opPop()
        opPush(x)
        opPush(x)
    else:
        errorHandler0()
    return
    
    
def exch():

    if enoughInSt2():
        x = opPop()
        b = opPop()
        opPush(x)
        opPush(b)
    else:
        print("error from exch():")
        errorHandler0()
    return
    

In [1485]:
# Define the dictionary manipulation operators in this cell: dictz, begin, end, def
# name the function for the def operator psDef because def is reserved in Python

def dictz(use = False):
    if use:
        if enoughInSt1():
            opPush("/--nostringval--")
        else:
            print("Error from dictz")
            errorHandler0()
    return
    
def begin(dy, st, use = False):
    if use:
        if enoughInSt1():
            x = opPop()
            tmp2 = x

            if type(tmp2) == str:
                if tmp2 == "/--nostringval--":
                    tmp2 = tmp2[1:]
                if tmp2 == "--nostringval--":
                    tmp = {}
                    tmp[tmp2] = "--nostringval--"
                    dictstack.append((dy,len(dictstack),st,tmp))
                else:
                    print("Error from begin()")
                    errorHandler1(x)
        else:
            print("Error from begin()")
            errorHandler0()
    return
    
def end(use = False):
    if use:
        if not lookup("--nostringval--"):
            print("Error from end()")
            errorHandler0()
        else:
            i = len(dictstack) - 1
            while "--nostringval--" not in dictstack[i][3]:
                dictPop()
                i -= 1
            dictPop()
            
    return
    
def psDef(dy=0,st=0):
    if enoughInSt2():
        x = opPop()
        b = opPop()
        define(b,x,dy,st)
    else:
        print("Error from psDef()")
        return False
    return True
    

In [1486]:
# Define the printing operators in this cell: =, stack
# Pick a good name for the code implementing =

#=
def popAndPrint():
    if enoughInSt1():
        print(opPop())
    return

def stack():
    if len(opstack) > 0:
        for i in range(len(opstack) - 1, -1, -1):
            print(opstack[i])
    return

In [1487]:
# Define any other operators that I may have forgotten in this cell

def roll():
    if enoughInSt2():
        num2 = opPop()
        num = opPop()
        num2 = num2%num
        if abs(num) <= len(opstack):
            if num2 > 0:
                tmp = opstack[0-num:]
                tmp2 = tmp[0-num2:]
                tmp_init = tmp[0:0-num2]
                tmp_final = tmp2
                tmp_final.extend(tmp_init)
                opstack[0-num:] = tmp_final

            elif num2 < 0:
                tmp = opstack[0-num:]
                tmp2 = tmp[0: 0-num2]
                tmp_init = tmp[0-num2:]
                tmp_final = tmp_init
                tmp_final.extend(tmp2)
                opstack[0-num:] = tmp_final
        else:
            errorHandler2(num2, num)
    else:
        errorHandler2()
    return

def count():
    return len(opstack)

def index():
    if enoughInSt1():
        num = opPop()
        tmp = num + 1
        if len(opstack) >= tmp and len(opstack) > 0:
            opPush(opstack[0 - tmp])
        else:
            errorHandler1(num)
    else:
        print("Error input: violate the syntax for those one element")
    return

def clear():
    opstack[:] = []
    
def psIf(rule, dy, st):
    if enoughInSt2():
        x = opPop()
        b = opPop()
        if b == "true":
                interpret(x, rule, dy, st)
        elif b != "false":
            errorHandler(x,b)
            print("Error input: violate the syntax for those two elements")
    else:
        print("Error input: violate the syntax for those two elements")
        
def psElif(rule, dy, st):
    if enoughInSt2():
        x = opPop()
        b = opPop()
        y = opPop()
        if y == "true":
                interpret(b, rule, dy, st)
        elif y == "false":
                interpret(x, rule, dy, st)
        else:
            opPush(y)
            opPush(b)
            opPush(x)
            print("Error input: violate the syntax for those two elements")
    else:
        print("Error input: violate the syntax for those two elements")
        

In [1488]:
# Copy this cell to your sps.ipynb and write the necessary code; again write
# auxiliary functions if you need them. This will probably be the largest
# function of the whole project, but it will have a very regular and obvious
# structure if you've followed the plan of the assignment.
def interpret(code, rule=True, dy = 0, st=0): # code is a code array
    theBigBigxX = 0
    if code :
        while theBigBigxX < len(code):
            
                target = None
                
                if type(code[theBigBigxX]) == list or not lookup(code[theBigBigxX], st, rule):
                    if code[theBigBigxX] == "pop":
                        opPop()
                    elif code[theBigBigxX] == "add":
                        add()
                    elif code[theBigBigxX] == "sub":
                        sub()
                    elif code[theBigBigxX] == "mul":
                        mul()
                    elif code[theBigBigxX] == "div":
                        div()
                    elif code[theBigBigxX] == "gt":
                        gt()
                    elif code[theBigBigxX] == "lt":
                        lt()
                    elif code[theBigBigxX] == "eq":
                        eq()
                    elif code[theBigBigxX] == "and":
                        psAnd()
                    elif code[theBigBigxX] == "or":
                        psOr()
                    elif code[theBigBigxX] == "exch":
                        exch()
                    elif code[theBigBigxX] == "not":
                        psNot()
                    elif code[theBigBigxX] == "dup":
                        dup()
                    elif code[theBigBigxX] == "dictz":
                            dictz()
                    elif code[theBigBigxX] == "begin":
                            begin(dy, st)
                    elif code[theBigBigxX] == "end":
                            end()
                    elif code[theBigBigxX] == "def":
                        if psDef(dy, st):
                            target = dictstack[len(dictstack) - 1]
                    elif code[theBigBigxX] == "=":
                        popAndPrint()
                    elif code[theBigBigxX] == "stack":
                        stack()
                    elif code[theBigBigxX] == "roll":
                        roll()
                    elif code[theBigBigxX] == "count":
                        count()
                    elif code[theBigBigxX] == "index":
                        index()
                    elif code[theBigBigxX] == "clear":
                        clear()
                    elif code[theBigBigxX] == "if":
                        psIf(rule, dy, st)
                    elif code[theBigBigxX] == "ifelse":
                        psElif(rule, dy, st)
                    else:
                        opPush(code[theBigBigxX])
                    if not str(dy) in seq:
                        seq[str(dy)] = []
                    if target != None and not target in seq[str(dy)]:
                        seq[str(dy)].append(target)    
                    
                elif type(lookup(code[theBigBigxX], st, rule)) == tuple:
                    val = lookup(code[theBigBigxX], st, rule)
                    if type(val[0]) == list:
                        if rule:
  
                            opPush("/"+str(code[theBigBigxX]))
                            opPush(val[0])
                            if psDef(dy, val[1]):
                                target = dictstack[len(dictstack) - 1]
                    
                            if not str(dy) in seq:
                                seq[str(dy)] = []
                            if target != None and not target in seq[str(dy)]:
                                seq[str(dy)].append(target) 
                                
                        interpret(val[0],rule,dy+1, lookup(code[theBigBigxX])[1])
                            
                    else:
                        opPush(val[0])
                            
                elif lookup(code[theBigBigxX], st, rule) != False:
                    opPush(lookup(code[theBigBigxX], st, rule))
                theBigBigxX = theBigBigxX + 1
    else:
        print("Error call from main:")
        errorHandler0()

    return

In [1489]:
def interpreter(s, rule=True): # if rule == True, then it is static scoping;
                               # if rule == False, then it is dynamic scoping.
                               # default value of rule is True
    interpret(parse(tokenize(s)), rule)


In [ ]:
# Copy this cell to your sps.ipynb. Add tests of your own, each in its own cell, and discuss how they provide
# evidence that your interpreter is working correctly. 
opstack = []
dictstack = []
seq = {}
interpreter(
"""
/x 4 def
/g { x stack } def
/f { /x 7 def g } def
f

""",False)

In [1506]:
def output():

    od = sorted(seq)
    s = "=============="
    print(s)
    if len(opstack) > 0:
        for x in opstack:
            print(x)
    else:
        print("(--Empty--)")
    print(s)
    tmp = []
    tmp1 = []

    for x in od:
        
        y = seq[x]

        for i in y:
            if len(tmp) == 0 or dictstack[tmp1[len(tmp1) - 1][2]][0] == dictstack[i[2]][0]:
                tmp1.append(i)
                for k, v in i[3].items():
                    tmp.append(str(k)+" "+str(v))
            
            else:
                tmp.append("---- " + str(x) + " ---- " + str(dictstack[i[2]][0]) + " ----")
                tmp1.append(i)
                for k, v in i[3].items():
                    tmp.append(str(k)+" "+str(v))    
        if len(y) > 0:
            tmp.append("---- " + str(x) + " ---- " + str(dictstack[y[len(y) - 1][2]][0]) + " ----")
        else:
            tmp.append("---- " + str(x) + " ---- " + str(0) + " ----")

    for i in range(len(tmp)-1, -1, -1):
        x = tmp[i]
        print(x)

    print(s)
        
    return

In [ ]:
output()